In [1]:
import pandas as pd

data = pd.read_csv('./data/hero-network.csv')

heroes = [
    'MEDUSA/MEDUSALITH AM',
    'ARCLIGHT/PHILLIPA SO',
    'WOLVERINE/LOGAN ',
    'SCARLET WITCH/WANDA ',
    'ARAGORN',
    'OVERMIND/GROM',
    'BATTLEAXE/',
    'ION/',
    'PINK PEARL/'
]

relations = data[(data.hero1.isin(heroes)) & (data.hero2.isin(heroes))]

import matplotlib.pyplot as plt
import matplotlib

import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


/usr/local/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## What's the problem again?


In [2]:
def maxcut(assigned_nodes, edges):
    return sum(edges.apply(
        lambda edge: (1-assigned_nodes[edge['hero1']]*assigned_nodes[edge['hero2']])/2,
        axis=1))

In [3]:
edges = relations
edges['check_string'] = edges.apply(lambda row: ''.join(sorted([row['hero1'], row['hero2']])), axis=1)
edges = edges.drop_duplicates('check_string')
edges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 895 to 390162
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   hero1         13 non-null     object
 1   hero2         13 non-null     object
 2   check_string  13 non-null     object
dtypes: object(3)
memory usage: 416.0+ bytes



## Solving It Classically


In [4]:
single_group = { name: -1 for name in heroes }
single_group

{'MEDUSA/MEDUSALITH AM': -1,
 'ARCLIGHT/PHILLIPA SO': -1,
 'WOLVERINE/LOGAN ': -1,
 'SCARLET WITCH/WANDA ': -1,
 'ARAGORN': -1,
 'OVERMIND/GROM': -1,
 'BATTLEAXE/': -1,
 'ION/': -1,
 'PINK PEARL/': -1}

In [5]:
maxcut(single_group, edges)

0.0

In [6]:
from random import seed, random
seed(None)

random_groups = { name: -1 if round(random()) == 0 else 1 for name in heroes }
random_groups

{'MEDUSA/MEDUSALITH AM': 1,
 'ARCLIGHT/PHILLIPA SO': -1,
 'WOLVERINE/LOGAN ': -1,
 'SCARLET WITCH/WANDA ': -1,
 'ARAGORN': -1,
 'OVERMIND/GROM': 1,
 'BATTLEAXE/': 1,
 'ION/': 1,
 'PINK PEARL/': -1}

In [7]:
maxcut(random_groups, edges)

8.0

In [8]:
def search(group, open_heroes):
    best = (0, None)
    
    if len(open_heroes) > 0:
        new = dict(group)
        new[open_heroes[0]] = new[open_heroes[0]]*(-1)
        
        result1 = search(group, open_heroes[1:])
        result2 = search(new, open_heroes[1:])
        
        best = result1 if result1[0] > result2[0] else result2
    else:
        return (maxcut(group, edges), group)
            
    return best

In [9]:
search(single_group, heroes)

(9.0,
 {'MEDUSA/MEDUSALITH AM': 1,
  'ARCLIGHT/PHILLIPA SO': 1,
  'WOLVERINE/LOGAN ': -1,
  'SCARLET WITCH/WANDA ': 1,
  'ARAGORN': 1,
  'OVERMIND/GROM': -1,
  'BATTLEAXE/': 1,
  'ION/': -1,
  'PINK PEARL/': -1})